In [3]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from importlib import reload
from PATH import data_path
from utils import *
from Model import *
import logger
import utils
import re
%matplotlib widget

from sklearn.preprocessing import scale

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RF

In [16]:
reload(logger)

<module 'logger' from '/home/ZwZ/script/HER2_prediction/logger.py'>

In [6]:
log_dir = 'PR_result/'

log_path = list(filter(lambda x: 'log' in x,os.listdir(log_dir)))
log_path = [os.path.join(log_dir,file) for file in log_path]

log_path

In [22]:
# test set take up twice more data than val set
def show_best(mlog):
    Mm_sum=np.add(mlog.val_f1auc_sum,3*mlog.test_f1auc_sum)
    print('raw log norm erbb2 \n')
    print(np.argmax(Mm_sum,axis=0))

# data

In [9]:
DF,y = get_known_sample(data_path,'PR')
unknown_df = get_unknown_sample(data_path,'PR')

In [28]:
datas = triple_preprocess(DF)
data_pre = triple_preprocess(unknown_df)

scale by gene ERBB2 in column 13739
scale by gene ERBB2 in column 13739


/home/ZwZ/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:190: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


## logger

In [96]:
RF_log=logger.Mlog(log_path[2],interval=25+16,seed=False)

# compare the evaluation

## LR

In [98]:
show_best(LR_log)

raw log norm erbb2

[ 81 540 540 328]


In [63]:
LR_log.performance_of(81)

,V_F1,V_AUCROC,T_F1,T_AUCROC
PREPROCESS,,,,
raw data,0.97521,0.99762,0.94902,0.93818
log data,0.99174,1.00000,0.96296,0.95731
norm data,0.99174,1.00000,0.95720,0.95392
erbb2 data,0.89474,0.90476,0.84342,0.80678


In [99]:
LR_seed = LR_log.seed_of(81)

In [119]:
set_ls,idx_ls = spliting_datas_by_the_same(datas,y,True,**{'idx':y,'random_state':LR_seed})

------------------------------------
Data Set    Raio of positive Sample
------------------------------------
 whole	 | 	 0.7731568998109641
 train	 | 	 0.7717717717717718
 val	 | 	 0.7972972972972973
 test	 | 	 0.7704402515723271
------------------------------------


In [123]:
LR_model = single_model(DataSet=set_ls[1],model=LR,**LR_arg)

In [128]:
LR_model.get_metrics(1,True)[0:2]

(0.9500698909700867, 0.943089430894309)

## LDA log

In [73]:
LDA_log = logger.Mlog(log_path[0])

In [92]:
show_best(LDA_log)

raw log norm erbb2

[767 993 518  80]


In [76]:
LDA_log.performance_of(993)

,V_F1,V_AUCROC,T_F1,T_AUCROC
PREPROCESS,,,,
raw data,0.96667,0.96595,0.97456,0.96327
log data,0.97561,0.98865,0.97476,0.96941
norm data,0.95798,0.94956,0.96047,0.93756
erbb2 data,0.73077,0.67528,0.69880,0.71176


In [177]:
LDA_seed=LDA_log.seed_of(767)

In [178]:
set_ls,idx_ls = spliting_datas_by_the_same(datas,y,True,**{'idx':y,'random_state':LDA_seed})

------------------------------------
Data Set    Raio of positive Sample
------------------------------------
 whole	 | 	 0.7731568998109641
 train	 | 	 0.7612612612612613
 val	 | 	 0.8378378378378378
 test	 | 	 0.7830188679245284
------------------------------------


In [179]:
LDA_model = single_model(DataSet=set_ls[1],model=LDA)

## RF log

In [23]:
RF_log = logger.Mlog(log_path[1],seed=True)

In [24]:
show_best(RF_log)

raw log norm erbb2 

[307 621 514 831]


In [25]:
RF_seed =  RF_log.all_seed[[307,621,514,831]]

In [26]:
RF_seed

array([2472636, 5719515,  490789, 9714761])

In [29]:
Set_ls = []
for i in range(len(RF_seed)):
    seed = RF_seed[i]
    set_ls,idx_ls = spliting_datas_by_the_same(datas,y,True,**{'idx':y,'random_state':seed})
    Set_ls.append(set_ls[i])

------------------------------------
Data Set    Raio of positive Sample
------------------------------------
 whole	 | 	 0.671090047393365
 train	 | 	 0.6566265060240963
 val	 | 	 0.6891891891891891
 test	 | 	 0.6971608832807571
------------------------------------
------------------------------------
Data Set    Raio of positive Sample
------------------------------------
 whole	 | 	 0.671090047393365
 train	 | 	 0.6521084337349398
 val	 | 	 0.7162162162162162
 test	 | 	 0.7003154574132492
------------------------------------
------------------------------------
Data Set    Raio of positive Sample
------------------------------------
 whole	 | 	 0.671090047393365
 train	 | 	 0.6596385542168675
 val	 | 	 0.7432432432432432
 test	 | 	 0.6782334384858044
------------------------------------
------------------------------------
Data Set    Raio of positive Sample
------------------------------------
 whole	 | 	 0.671090047393365
 train	 | 	 0.6656626506024096
 val	 | 	 0.6756756756756757

In [30]:
model_ls = [single_model(DataSet=Set_ls[i],model=RF,**RF_arg) for i in range(4)]

# predict

In [32]:
predict = np.array([model_ls[i]._model.predict(data_pre[i]) for i in range(4)]).T

In [33]:
predict_sum=np.sum(predict,axis=1)

In [35]:
result_dic ={4:1,3:1,2:np.random.randint(0,2,1).item(),1:0,0:0}

In [42]:
predict_sum

array([0, 0, 4, 3, 1, 3, 1, 3, 3, 4, 3, 2, 0, 2, 2, 0, 4, 4, 0, 1, 4, 4,
       4, 2, 3, 0, 3, 2, 2, 3, 4, 3, 4, 0, 4, 0, 4, 4, 4, 1, 4, 1, 3, 0,
       0, 2, 0, 4, 0, 4, 3, 3, 4])

In [36]:
final_result = [result_dic[i] for i in predict_sum]

In [39]:
pre_DF=pd.DataFrame(final_result,index=unknown_df.index,columns=['PR_predict'])

In [40]:
pre_DF.to_csv('PR_result/PR_predict_result.csv')